# Guided exercise 7: Higher order functions (II)





**Exercise 1.** Use recursion to create your own `sum` function that adds all the elements of a list

In [2]:
add::(Num a)=> [a] -> a
add [] = 0
add (x:xs) = x + add xs
-- Haskell suggests to use foldr

Line 3: Use foldr
Found:
add [] = 0
add (x : xs) = x + add xs
Why not:
add xs = foldr (+) 0 xs

In [20]:
-- Invoking the function
add [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

55

## Folds

`foldr f acc xs` and `foldl f acc xs` are similar functions, both apply a binary function over all the elements of a list plus an initial value, which is called the accumulator. The difference lies in the order of the parameters of the function: `f x acc` for `foldr` or `f acc x` for `foldl` and from where we start taking `x` from `xs`, from the head or from the last:

`foldr f acc x:y:z = f x (f y (f z acc))`

`foldl f acc x:y:z = f (f (f acc x) y) z` 

`foldr` is easier to be understood from a recursive point of view: `foldr f acc x:xs = f x (foldr f acc xs)`. Meanwhile `foldl` is easier seen as traversing the list applying the function between the accumulator and all its members in sequence.

In [21]:
foldr (-) 5 [1, 2, 3]
foldr (-) 5 [1, 2, 3] == 1 - (2 - (3 - 5))
foldl (-) 5 [1, 2, 3]
foldl (-) 5 [1, 2, 3] == ((5 - 1) - 2) - 3

-3

True

-1

True

If the function is commutative both behave the same

In [2]:
foldr (+) 4 [1, 2, 3] == 1 + (2 + (3 + 4))
foldl (+) 4 [1, 2, 3] == ((4 + 1) + 2) + 3

True

True

If it is not, the result will be in general different

In [3]:
foldr (-) 0 [1, 2, 3]
foldl (-) 0 [1, 2, 3]

2

-6

Even if sometimes, by chance, it can be the same

In [4]:
foldr (-) 4 [1, 2, 3]
foldl (-) 4 [1, 2, 3]

-2

-2

**Exercise 1.** Create `myMax xs` function that calculates the maximum of a list with a lambda and a fold

In [3]:
-- This lambda is conmmutative so we can use it in both folds, but we aware that for foldr the lambda should be \x acc and for foldl it should be \acc x
myMax:: Ord a => [a] -> a
myMax (x:xs) = foldr (\a acc -> if a > acc then a else acc) x xs
-- myMax (x:xs) = foldr max x xs

myMax':: Ord a => [a] -> a
myMax' (x:xs) = foldl (\acc b -> if acc > b then acc else b) x xs
-- myMax' (x:xs) = foldl max x xs

Line 5: Use max
Found:
if a > b then a else b
Why not:
max a bLine 8: Use max
Found:
if a > b then a else b
Why not:
max a b

In [2]:
myMax [3, 2, 1]
myMax [1, 2, 3]
myMax [1, 3, 2]
myMax' [3, 2, 1]
myMax' [1, 2, 3]
myMax' [1, 3, 2]

3

3

3

3

3

3

`foldl1` and `foldr1` are like `foldl` and `foldr` but the initial value is not needed, they start with the first or last element of the list, respectively (they are partial as they cannot be used on empty lists).

In [7]:
myMax'':: Ord a => [a] -> a
-- We could remove xs from here
myMax'' xs = foldl1 (\a b -> if a > b then a else b) xs

Line 3: Eta reduce
Found:
myMax'' xs = foldl1 (\ a b -> if a > b then a else b) xs
Why not:
myMax'' = foldl1 (\ a b -> if a > b then a else b)

In [8]:
myMax'' [3, 2, 1]
myMax'' [1, 2, 3]
myMax'' [1, 3, 2]

3

3

3

**Exercise 2.** Create a function `multiply xs` that multiplies all the elements of a list

In [9]:
multiply = foldl1 (*)

Line 1: Use product
Found:
foldl1 (*)
Why not:
product

In [10]:
multiply [1..5]
multiply [2]

120

2

**Exercise 3.** Create a function `bit2int xs` that given a list of 1's and 0's representing a binary number returns its decimal representation

In [38]:
-- Every time a new element is 'discovered' it is the bit at position 1, so all the previous ones are multiplied by 2
bin2int :: [Int] -> Int
bin2int  = foldl (\acc x -> x + 2*acc) 0

-- We can also do it with a foldr if we reverse the list. Notice the order of parameters in lambda
bin2int' :: [Int] -> Int
bin2int' xs = foldr (\x acc -> x + 2*acc) 0 $ reverse xs

In [39]:
bin2int [1,0,0]
bin2int [1,0,1,1]
bin2int [0,0,0]
bin2int []
bin2int' [1,0,0]
bin2int' [1,0,1,1]
bin2int' [0,0,0]
bin2int' []

4

11

0

0

4

11

0

0

# The `$` operator and function composition
The `$` operator is used to avoid parentheses in function application, it is like the space but it has the lowest precedence

In [11]:
even (1 + 2 + 3 + 4)
even $ 1 + 2 + 3 + 4

True

True

Function composition `.` is also used to reduce the number of parentheses

In [12]:
addAll::(Num a)=> [[a]] -> a
addAll x = sum (map sum x)

In [13]:
addAll'::(Num a)=> [[a]] -> a
addAll'= sum . map sum

In [14]:
addAll' [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10]]

55

Some combinations

In [15]:
-- To check if the sum of a list is even
-- Regular way
even (sum [1, 2, 3, 4])
-- With $ and .
even $ sum [1,2, 3, 4]
even . sum $ [1, 2, 3, 4]
(even . sum) [1, 2, 3, 4]

True

True

True

True

Use function composition to make things clearer, particularly in maps to avoid lambda functions

In [12]:
-- Function that states if the sums of the sublists of a list are even
evenSums :: (Integral a, Foldable t) => [t a] -> [Bool]
evenSums  = map (\ ys -> even $ sum ys)

evenSums' :: (Integral a, Foldable t) => [t a] -> [Bool]
evenSums' = map $ even . sum 

Line 2: Avoid lambda
Found:
\ ys -> even $ sum ys
Why not:
even . sum

In [17]:
evenSums [[1, 2],[3, 4],[5, 6, 7]]
evenSums' [[1, 2],[3, 4],[5, 6, 7]]

[False,False,True]

[False,False,True]